In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2
from fastai.vision import *
from pathlib import Path
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from comic.dataset import ImageTextDataset
import comic.training.transforms as T
from comic.training.engine import train_one_epoch, evaluate
import comic.utils
from comic.vis import draw_annotation, show_image
from comic.models.rcnn import get_model_box_detector, save_model

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    #if train:
    #    transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 3
# use our dataset and defined transformations
dataset = ImageTextDataset('../data/cleaned_dataset', get_transform(train=True), include_masks=False)
dataset_test = ImageTextDataset('../data/cleaned_dataset', get_transform(train=False), include_masks=False)

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-10])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-10:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=comic.utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=comic.utils.collate_fn)

print(f'Training dataset {len(dataset)} images')
print(f'Validation dataset {len(dataset_test)} images')


In [ ]:
idx = 40
print(len(dataset))
image, targets = dataset[idx]
#image = list(img.cuda() for img in image)
#show_image(targets['masks'][0][None, :, :], figsize=(10,10))
draw_annotation(image, targets, figsize=(10,10))
print(targets['boxes'])

In [ ]:
# get the model using our helper function
model = get_model_box_detector(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

In [ ]:
idx = 1
image, targets = dataset_test[idx]
#image = list(img.cuda() for img in image)
#show_image(targets['masks'][0][None, :, :], figsize=(10,10))
draw_annotation(image, targets, figsize=(10,10))
print(targets['boxes'])


In [ ]:
outputs = model(image[None, ...].cuda())
print(outputs[0]['scores'])
draw_annotation(image, outputs[0], figsize=(10,10))

In [ ]:
print(image.shape)

In [ ]:
#save_model(model, 'box_detector')